In [47]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint

import pandas as pd

energy_cancer = pd.read_csv("Machine Learning Dataset-Cancer&Energy_dropCancerColumns.csv")
energy_cancer.head()
energy_cancer.drop(columns=['Unnamed: 0', 'State Abbreviation'], inplace=True)
energy_cancer.head()

,2011(C),2012(C),2013(C),2014(C),2015(C),2016(C),2017(C),2018(C),2019(C),2020(C),...,2012(W),2013(W),2014(W),2015(W),2016(W),2017(W),2018(W),2019(W),2020(W),LungBronchus_Classifer
0,15481,15521,14819,18225,19511,16613,16376,17325,17620,18527,...,356,1386,1445,1487,1562,1305,1410,1269,1127,0
1,651032,547004,565051,575912,494311,410168,378890,377152,317180,256687,...,0,0,0,0,0,0,0,0,0,1
2,306119,296732,327099,339214,226889,246437,267591,304145,239798,161980,...,0,0,0,0,0,0,0,0,0,1
3,459909,420570,454865,447849,385822,323878,334512,331492,257727,156821,...,5059,4291,4451,4210,4998,5248,4824,4933,5647,1
4,55264,43832,38151,39486,30967,32077,33663,33256,30886,28034,...,92821,122335,123546,113906,124661,118074,127590,122228,119083,0


In [48]:
energy_cancer.nunique()

2011(C)                   51
2012(C)                   51
2013(C)                   51
2014(C)                   51
2015(C)                   51
                          ..
2017(W)                   43
2018(W)                   43
2019(W)                   43
2020(W)                   43
LungBronchus_Classifer     2
Length: 101, dtype: int64

In [49]:
# Split our preprocessed data into our features and target arrays
y = energy_cancer.LungBronchus_Classifer.values
X = energy_cancer.drop(columns="LungBronchus_Classifer").values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [50]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [51]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=200, activation='relu', input_dim=100))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=50, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 200)               20200     
                                                                 
 dense_7 (Dense)             (None, 50)                10050     
                                                                 
 dense_8 (Dense)             (None, 1)                 51        
                                                                 
Total params: 30301 (118.36 KB)
Trainable params: 30301 (118.36 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [52]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [59]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50, validation_data=(X_test, y_test))

Epoch 1/50
2/2 [==============================] - 0s 194ms/step - loss: 0.0605 - accuracy: 1.0000 - val_loss: 219520.2812 - val_accuracy: 0.6154
Epoch 2/50
2/2 [==============================] - 0s 116ms/step - loss: 0.0578 - accuracy: 1.0000 - val_loss: 232215.1562 - val_accuracy: 0.6154
Epoch 3/50
2/2 [==============================] - 0s 51ms/step - loss: 0.0582 - accuracy: 1.0000 - val_loss: 238898.1406 - val_accuracy: 0.6154
Epoch 4/50
2/2 [==============================] - 0s 71ms/step - loss: 0.0582 - accuracy: 1.0000 - val_loss: 239718.5000 - val_accuracy: 0.6154
Epoch 5/50
2/2 [==============================] - 0s 73ms/step - loss: 0.0551 - accuracy: 1.0000 - val_loss: 238875.5781 - val_accuracy: 0.6154
Epoch 6/50
2/2 [==============================] - 0s 64ms/step - loss: 0.0576 - accuracy: 1.0000 - val_loss: 244986.5156 - val_accuracy: 0.6154
Epoch 7/50
2/2 [==============================] - 0s 66ms/step - loss: 0.0597 - accuracy: 1.0000 - val_loss: 264510.6250 - val_accurac

In [60]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

1/1 - 0s - loss: 1.0069 - accuracy: 0.6154 - 22ms/epoch - 22ms/step
Loss: 1.0068869590759277, Accuracy: 0.6153846383094788
